### Links
> IBGE censo https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html
------------------------
> Prefeitura SP projeções https://www.prefeitura.sp.gov.br/cidade/secretarias/licenciamento/desenvolvimento_urbano/dados_estatisticos/info_cidade/demografia/index.php?p=260265


In [1]:
import $ivy.`org.apache.spark::spark-core:3.1.2`
import $ivy.`org.apache.spark::spark-sql:3.1.2`
import $ivy.`org.apache.hadoop:hadoop-client:3.1.2`
import $ivy.`org.apache.hadoop:hadoop-aws:3.1.2`
import $ivy.`com.amazonaws:aws-java-sdk-s3:1.11.375`
import $ivy.`com.amazonaws:aws-java-sdk-core:1.11.375`


import $ivy.$                                   

import $ivy.$                                  

import $ivy.$                                      

import $ivy.$                                   

import $ivy.$                                       

import $ivy.$                                         


In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = {
    import org.apache.log4j.{Level, Logger}
    Logger.getLogger("org").setLevel(Level.OFF)
      SparkSession.builder()
        .master("local[3]")
        .config("spark.sql.warehouse.dir", "s3://fernando-bontorin-censo")
        .getOrCreate()
}

import spark.implicits._
import org.apache.spark.sql.expressions.Window

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@2c2a58db
import spark.implicits._

import org.apache.spark.sql.expressions.Window

In [3]:
val projecoes = spark.read.options(
    Map("inferSchema" -> "true", "header" -> "true", "delimiter" -> ";", "encoding" -> "UTF-8")
).csv("s3a://fernando-bontorin-censo/custom_IBGE_projecoes_populac_2008_10573.CSV")

projecoes.printSchema

root
 |-- Distritos: string (nullable = true)
 |-- 2017: string (nullable = true)
 |-- 2018: string (nullable = true)
 |-- 2019: string (nullable = true)
 |-- 2020: string (nullable = true)
 |-- 2021: string (nullable = true)



projecoes: DataFrame = [Distritos: string, 2017: string ... 4 more fields]

In [4]:
def stringNumber(s: String) = regexp_replace(col(s), lit(" "), lit("")).cast("long").as(s)

val projecoesPorAno = projecoes
.select(col("Distritos"), stringNumber("2017"), stringNumber("2018"), stringNumber("2019"), stringNumber("2020"), stringNumber("2021"))
.flatMap(row=> Seq(
    (row.getAs[String]("Distritos"), row.getAs[Long]("2017"), 2017),
    (row.getAs[String]("Distritos"), row.getAs[Long]("2018"), 2018),
    (row.getAs[String]("Distritos"), row.getAs[Long]("2019"), 2019),
    (row.getAs[String]("Distritos"), row.getAs[Long]("2020"), 2020),
    (row.getAs[String]("Distritos"), row.getAs[Long]("2021"), 2021)
)).toDF("distrito", "populacao_total", "ano")

projecoesPorAno.show(6)

+----------+---------------+----+
|  distrito|populacao_total| ano|
+----------+---------------+----+
|Bela Vista|          72182|2017|
|Bela Vista|          72543|2018|
|Bela Vista|          72897|2019|
|Bela Vista|          73243|2020|
|Bela Vista|          73590|2021|
|Bom Retiro|          39058|2017|
+----------+---------------+----+
only showing top 6 rows



defined function stringNumber
projecoesPorAno: DataFrame = [distrito: string, populacao_total: bigint ... 1 more field]

In [5]:
val censo = spark.read.options(
    Map("inferSchema" -> "true", "header" -> "true", "delimiter" -> ";", "encoding" -> "UTF-8")
).csv("s3a://fernando-bontorin-censo/Base_informacoes_setores2010_sinopse_SP_RM_Sao_Paulo_Santos.csv")


censo: DataFrame = [Cod_setor: bigint, Cod_Grandes Regiões: int ... 172 more fields]

In [6]:
def censo_isSPCapital: Column = when(col("Cod_municipio") === 3550308, true).otherwise(false)
def freeX(s: String): Column = when(lower(trim(col(s))) === "x", 0).otherwise(col(s).cast("int")).as(s)

val capital_censos = censo.where(censo_isSPCapital).select(
    Seq(col("Nome_do_distrito").as("distrito")) ++
    censo.columns.filter(x=>x.startsWith("V")).map(s=>freeX(s)):_*
)

defined function censo_isSPCapital
defined function freeX
capital_censos: DataFrame = [distrito: string, V001: int ... 153 more fields]

In [7]:
val windowSpecAgg = org.apache.spark.sql.expressions.Window.partitionBy(col("distrito"))

windowSpecAgg: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@6490684c

In [8]:
val distritos_somatoria = capital_censos.select(
    Seq(col("distrito")) ++ censo.columns.filter(x=>x.startsWith("V")).map(c=>sum(col(c)).over(Window.partitionBy(col("distrito"))).as(c)):_*
).dropDuplicates

distritos_somatoria: Dataset[Row] = [distrito: string, V001: bigint ... 153 more fields]

In [9]:
def idadeRatio(c: Column) = when(c > 0, c/col("V014") ).otherwise(0.0)

def clusterIdadeRatio(colsNames: String*) = idadeRatio(colsNames.map(col).reduce(_ + _))

defined function idadeRatio
defined function clusterIdadeRatio

In [10]:
val distritos_perfis_idade = distritos_somatoria.select(
    col("distrito"),
    clusterIdadeRatio("V032", "V033", "V034", "V035", "V036").as("razao_residentes_0_4"),
    clusterIdadeRatio("V037", "V038", "V039", "V040", "V041").as("razao_residentes_5_9"),
    clusterIdadeRatio("V042", "V043", "V044", "V045", "V046").as("razao_residentes_10_14"),
    clusterIdadeRatio("V047", "V048", "V049", "V050", "V051").as("razao_residentes_15_19"),
    clusterIdadeRatio("V052", "V053", "V054", "V055", "V056").as("razao_residentes_20_24"),
    idadeRatio(col("V057")).as("razao_residentes_25_29"),
    idadeRatio(col("V058")).as("razao_residentes_30_34"),
    idadeRatio(col("V059")).as("razao_residentes_35_39"),
    idadeRatio(col("V060")).as("razao_residentes_40_44"),
    idadeRatio(col("V061")).as("razao_residentes_45_49"),
    idadeRatio(col("V062")).as("razao_residentes_50_54"),
    idadeRatio(col("V063")).as("razao_residentes_55_59"),
    idadeRatio(col("V064")).as("razao_residentes_60_64"),
    idadeRatio(col("V065")).as("razao_residentes_65_69"),
    idadeRatio(col("V066")).as("razao_residentes_70_74"),
    idadeRatio(col("V067")).as("razao_residentes_75_79"),
    idadeRatio(col("V068")).as("razao_residentes_80_84"),
    idadeRatio(col("V069")).as("razao_residentes_85_89"),
    idadeRatio(col("V070")).as("razao_residentes_90_94"),
    idadeRatio(col("V071")).as("razao_residentes_95_99"),
    idadeRatio(col("V072")).as("razao_residentes_100")
)

distritos_perfis_idade: DataFrame = [distrito: string, razao_residentes_0_4: double ... 20 more fields]

In [11]:
distritos_perfis_idade.show(1)

+--------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------+
|distrito|razao_residentes_0_4|razao_residentes_5_9|razao_residentes_10_14|razao_residentes_15_19|razao_residentes_20_24|razao_residentes_25_29|razao_residentes_30_34|razao_residentes_35_39|razao_residentes_40_44|razao_residentes_45_49|razao_residentes_50_54|razao_residentes_55_59|razao_residentes_60_64|razao_residentes_65_69|razao_residentes_70_74|razao_residentes_75_79|razao_residentes_80_84|razao_residentes_85_89|razao_residentes_90_94|razao_residentes_95_99|razao_residentes_100|
+--------+--------------

In [13]:
val distrito_tamanho = spark.read.options(
    Map("inferSchema" -> "true", "header" -> "true", "delimiter" -> ";", "encoding" -> "UTF-8")
).csv("s3a://fernando-bontorin-censo/tamanho_regiao_sp_distritos.csv")
.select(col("distrito"), col("regiao_ha").as("tamanho_em_ha"), col("regiao_metros").as("tamanho_em_m2"), col("regiao_km").as("tamanho_em_km2"))

distrito_tamanho: DataFrame = [distrito: string, tamanho_em_ha: double ... 2 more fields]

In [14]:
distrito_tamanho.show(2)

+----------+-------------+-------------+--------------+
|  distrito|tamanho_em_ha|tamanho_em_m2|tamanho_em_km2|
+----------+-------------+-------------+--------------+
|Aricanduva|        660.0|      6600000|           6,6|
|    Carrão|        750.0|      7500000|           7,5|
+----------+-------------+-------------+--------------+
only showing top 2 rows



In [15]:
def distrito_padronizado = udf((s: String) =>
                               s.trim.toLowerCase
                               .replace("ã", "a")
                               .replace("â", "a")
                               .replace("á", "a")
                               .replace("é", "e")
                               .replace("í", "i")
                               .replace("ô", "o")
                               .replace("ó", "o")
                               .replace("ú", "u")
                               .replace("ç", "c"))

defined function distrito_padronizado

In [16]:
val dfCenso = projecoesPorAno.withColumn("distrito_pr", distrito_padronizado(col("distrito")))
.join(distrito_tamanho.withColumn("distrito_pr", distrito_padronizado(col("distrito"))).drop("distrito"), Seq("distrito_pr"), "left")
.join(distritos_perfis_idade.withColumn("distrito_pr", distrito_padronizado(col("distrito"))).drop("distrito"), Seq("distrito_pr"), "left")

dfCenso: DataFrame = [distrito_pr: string, distrito: string ... 26 more fields]

In [17]:
dfCenso.printSchema
dfCenso.show(2)

root
 |-- distrito_pr: string (nullable = true)
 |-- distrito: string (nullable = true)
 |-- populacao_total: long (nullable = false)
 |-- ano: integer (nullable = false)
 |-- tamanho_em_ha: double (nullable = true)
 |-- tamanho_em_m2: integer (nullable = true)
 |-- tamanho_em_km2: string (nullable = true)
 |-- razao_residentes_0_4: double (nullable = true)
 |-- razao_residentes_5_9: double (nullable = true)
 |-- razao_residentes_10_14: double (nullable = true)
 |-- razao_residentes_15_19: double (nullable = true)
 |-- razao_residentes_20_24: double (nullable = true)
 |-- razao_residentes_25_29: double (nullable = true)
 |-- razao_residentes_30_34: double (nullable = true)
 |-- razao_residentes_35_39: double (nullable = true)
 |-- razao_residentes_40_44: double (nullable = true)
 |-- razao_residentes_45_49: double (nullable = true)
 |-- razao_residentes_50_54: double (nullable = true)
 |-- razao_residentes_55_59: double (nullable = true)
 |-- razao_residentes_60_64: double (nullable = 

In [18]:
dfCenso
.coalesce(1)
.write
.mode(SaveMode.Overwrite)
.options(
    Map("header" -> "true", "delimiter" -> ",", "encoding" -> "UTF-8")
)
.csv("s3a://fernando-bontorin-censo/distritos_censo_projecoes_e_demografias")